In [2]:
import numpy as np
import pandas as pd
# how to merge two dataframes and do inner/outer/left joins
# how to do analysis on dataframes
# creating a csv file
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
monthwise_revenue_train = pd.read_csv("patient_monthwise_revenue_train.csv")
appointments_train     = pd.read_csv("physio_appts_train.csv")
diagnosis_train        = pd.read_csv("physio_diagnosis_train.csv")
classified_train       = pd.read_csv("patient_train_classified.csv")

In [106]:
monthwise_revenue_train.columns

Index(['patient_id', 'visit_month_year', 'service_id', 'visits_count', 'city',
       'revenue', 'ref_type', 'ref_name', 'ref_source', 'service_name', 'FVD',
       'FVM', 'FVS', 'approx_age', 'gender', 'LVD', 'brand', 'visits_required',
       'diagnosis', 'avg_nps'],
      dtype='object')

In [107]:
visit_count = monthwise_revenue_train.groupby(['patient_id']).count()
visit_count.head()

,visit_month_year,service_id,visits_count,city,revenue,ref_type,ref_name,ref_source,service_name,FVD,FVM,FVS,approx_age,gender,LVD,brand,visits_required,diagnosis,avg_nps
patient_id,,,,,,,,,,,,,,,,,,,
10020,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10293,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10381,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
10388,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
10611,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [108]:
# rev suffix means its a monthly revenue train file
visit_count_rev = visit_count['visits_count' ]
visit_count_rev.head()

patient_id
10020     1
10293     1
10381     4
10388    11
10611     3
Name: visits_count, dtype: int64

In [109]:
visit_count_rev = visit_count_rev.to_frame()

In [110]:
# commented, prevent multiple file saves
#visit_count_rev = visit_count_rev.to_csv('entries_count_rev.csv')

In [111]:

#lets handle a categorical shit -> City , deal with codes here :) 

In [112]:
mrt_df = monthwise_revenue_train.city.to_frame()

In [113]:
city_rev_df = mrt_df["city"].astype('category')

In [114]:
city_rev_df = city_rev_df.cat.codes.to_frame()
patient_id_df = monthwise_revenue_train.patient_id.to_frame()


In [115]:
concat1 = pd.concat([patient_id_df, city_rev_df], axis = 1)

In [116]:
concat1 = concat1.groupby('patient_id').max()


In [117]:
concat1.columns

Index([0], dtype='object')

In [118]:
# commented, prevent multiple file saves
#concat1.to_csv('patient_city_rev.csv')

In [119]:
rev_sum = monthwise_revenue_train.groupby('patient_id')

In [120]:
rev_sum = rev_sum.sum()

In [121]:
# commented, prevent multiple file saves
#rev_sum.to_csv('revennue_visits_required_rev.csv')

In [122]:
monthwise_revenue_train.isnull().sum()

patient_id             0
visit_month_year       0
service_id             0
visits_count           0
city                   0
revenue                0
ref_type               0
ref_name               0
ref_source             0
service_name        1223
FVD                    0
FVM                    0
FVS                    0
approx_age             0
gender                33
LVD                    0
brand                  0
visits_required        0
diagnosis           1733
avg_nps                0
dtype: int64

In [123]:
# to - do 
# codify ref_type
# codify ref_name
# codify ref_source
# cofidy service_name
# FVD-LVD
# drop FVS
# unique age 
# codify gender
# codify brand
# convert avg_np to NAN , see what to do here
# as we can see in the block above, null values exist for service_name, gender, diagnosis - what to do there?

In [124]:
#check number of unique values for ref_type
monthwise_revenue_train.ref_type.unique()

array(['B2C', 'B2B'], dtype=object)

In [125]:
ref_type_rev_cat = monthwise_revenue_train["ref_type"].astype('category')

In [126]:
ref_type_rev_cat_code = ref_type_rev_cat.cat.codes

In [127]:
ref_type_rev_cat_code = ref_type_rev_cat_code.to_frame()

In [128]:
# check the entries for patient_id_df
patient_id_df.head()

,patient_id
0,10020
1,10293
2,10381
3,10381
4,10381


In [129]:
# check the entries for ref_type_rev_cat_code
ref_type_rev_cat_code.head()

,0
0,1
1,1
2,1
3,1
4,1


In [130]:
patient_ref_type = pd.concat([patient_id_df, ref_type_rev_cat_code], axis = 1)
patient_ref_type.head()
patient_ref_type = patient_ref_type.groupby('patient_id')

In [131]:
# check the ref type data.
patient_ref_type = patient_ref_type.max()
# get the csv

#patient_ref_type = patient_ref_type.to_csv('patient_ref_type.csv')


In [132]:
# check the number of unique entries in ref_name
monthwise_revenue_train.ref_name.nunique()

310

In [133]:
#codify the ref_name
ref_name_codes = monthwise_revenue_train["ref_name"].astype('category')

In [134]:
ref_name_codes = ref_name_codes.cat.codes.to_frame()

In [135]:
ref_name_codes.head()

,0
0,215
1,111
2,215
3,215
4,215


In [136]:
# merge the dataframes
patient_ref_names = pd.concat([patient_id_df, ref_name_codes], axis = 1)

In [137]:
# check the entries before converting to csv
patient_ref_names.head()

,patient_id,0
0,10020,215
1,10293,111
2,10381,215
3,10381,215
4,10381,215


In [138]:
# get the unique patient_ids
patient_ref_names = patient_ref_names.groupby('patient_id')
patient_ref_names = patient_ref_names.max()

# get the csv
#patient_ref_names = patient_ref_names.to_csv('patient_ref_names.csv')

In [139]:
# codify ref_source
# get unique entries for ref_source

monthwise_revenue_train.ref_source.nunique()

11

In [140]:
#codify ref_source shit
patient_id_ref_source_code = monthwise_revenue_train["ref_source"].astype('category')
# get the count
patient_id_ref_source_code = patient_id_ref_source_code.cat.codes

In [141]:
patient_id_ref_source_code = patient_id_ref_source_code.to_frame()

In [142]:
# check sample entries
patient_id_ref_source_code.head()

,0
0,5
1,4
2,5
3,5
4,5


In [143]:
# merge with patient id data
patient_id_ref_source = pd.concat([patient_id_df, patient_id_ref_source_code], axis = 1)

In [144]:
# check the entries for ref_source code
patient_id_ref_source.head()

,patient_id,0
0,10020,5
1,10293,4
2,10381,5
3,10381,5
4,10381,5


In [145]:
patient_id_ref_source = patient_id_ref_source.groupby('patient_id')

In [146]:
patient_id_ref_source = patient_id_ref_source.max()

In [4]:
#patient_id_ref_source = patient_id_ref_source.to_csv('patient_id_ref_source.csv')

In [5]:
monthwise_revenue_train.service_name.unique()

array(['P', 'D', 'N', 'Phle', nan, 'Tele', ' ', 'KNR', 'LTN', 'NT', 'CO',
       'Staff'], dtype=object)

In [8]:
monthwise_revenue_train.service_name.value_counts()

P        78879
Phle      1745
N          720
D          394
NT         157
LTN        106
KNR         47
Tele        21
Staff       13
CO           1
             1
Name: service_name, dtype: int64